<a href="https://colab.research.google.com/github/jsansao/idl/blob/main/Licao7_FashionMNIST_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

Esse notebook foi adaptado de https://github.com/lmoroney/dlaicourse



In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


# Melhorando a acúracia usando Convoluções 

Na lição 6, podemos verificar o desempenho das redes neurais profundas (DNN) com 3 camadas (camada de entrada, camada de saída e camada oculta). 

Experimentamos com diferentes parâmetros para verificar o desempenho da rede neural para esse conjunto de dados (Fashion MNIST). 

Abaixo, o código básico da lição para comparação. 



In [2]:
import tensorflow as tf
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images / 255.0
test_images=test_images / 255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=5)

test_loss = model.evaluate(test_images, test_labels)

4431872/4422102 [==============================] - 0s 0us/step
Epoch 1/5
1875/1875 [==============================] - 8s 2ms/step - loss: 0.4993 - accuracy: 0.8255
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3734 - accuracy: 0.8647
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3363 - accuracy: 0.8768
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3119 - accuracy: 0.8844
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.3471 - accuracy: 0.8741


Note que acúracia de treinamento está na faixa dos 89% e acurácia de validação na faixa de 87%. 

Com adição de camadas de convolução e pooling antes das camadas densas, formamos a chamada CNN (convolutional neural network). 

Pelas características de realçamento dos filtros convolucionais, a adição destas camadas tem o potencial de aumentar a acúracia dos classificadores, como veremos a seguir. 

No trecho a seguir, adicionou-se 2 camadas convolucionais na rede. Compare a acúracia com a rede anterior (totalmente conectada). 

Para acelerar o processamento, considere alterar o runtime do ambiente para GPU (veja em Runtime/Change Runtime Type/ Hardware Accelerator/GPU).
  

In [3]:
import tensorflow as tf
print(tf.__version__)
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(training_images, training_labels, epochs=5)
test_loss = model.evaluate(test_images, test_labels)


2.8.0
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 1600)              0         
                                                                 
 dense_2 (Dense)             (None, 128)        

In [4]:
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images2=training_images.reshape(60000, 28, 28, 1)



print(training_images.shape)
print(training_images2.shape)


(60000, 28, 28)
(60000, 28, 28, 1)


Note que a acúracia sobe para 93% no conjunto de treinamento e 91% no conjunto de testes. 

Tente agora rodar por mais épocas (~20), repare o aumento da acúracia de treinamento mas o mesmo não ocorre para a validação. Esse é o fenômeno do *overfitting*.

*Overfitting* ocorre quando a rede se especializa excessivamente no conjunto de treinamento, ficando menos efetiva para outros conjuntos de dados. 


In [5]:
import tensorflow as tf
print(tf.__version__)
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(training_images, training_labels, epochs=20)
test_loss = model.evaluate(test_images, test_labels)


2.8.0
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 64)        36928     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 1600)              0         
                                                                 
 dense_4 (Dense)             (None, 128)        


Note que o primeiro passo é transformar os dados de entrada em um tensor. Originalmente, tem-se uma lista de 60000 itens com 28x28x1, passa-se a um tensor 60000x28x28x1. 

O passo é necessário para o uso da camada de convolução. 

```
import tensorflow as tf
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0
```


Próximo passo é definir o modelo. 

Na camada inicial convolucional:

1. O número de convoluções da primeira camada é puramente arbitrário. Um bom palpite inicial é 32. 

2. Tamanho da matriz de convolução (por exemplo 3x3)

3. Função de ativação (neste caso reLu)

4. Formato de entrada: neste caso (28,28,1), imagens 28x28 em tons de cinza (uma dimensão)

O próximo passo é seguir a camada de convolução por uma camada de "MaxPooling", que tem como objetivo comprimir a imagem filtrada, isto é, diminuir o número de parâmetros entre uma etapa e outra. 

Ao especificar  (2,2) no Pooling, o tamanho vai para um quarto do original. Dividindo a imagem total em arranjos 2x2 de pixels, o método escolhe 1 dos 4 pixels, o de maior módulo. 

O comando ``model.summary()`` descreve o tamanho e forma da rede neural. A cada camada de pooling, o tamanho da imagem é reduzido. 


```
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
```


Adicionando mais uma camada de convulação/pooling


```
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2)
```


Achatando a saída, para se ter o mesmo tipo de saída da rede convencional

```
  tf.keras.layers.Flatten(),
```




Aqui, colocou-se 128 neurônios na camada densa e 10 neurônios na camada de saída. 


```
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
```




Finalmente a compilação, treinamento e avaliação do modelo. 


```
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=5)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)
```


## Exercícios exploratórios


1. Varie o número de convoluções. Mude N=32 para 16 ou 64. Qual é o impacto na acurácia e tempo de treinamento?

N=32


In [6]:
import tensorflow as tf
print(tf.__version__)
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),  
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=10)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)

2.8.0
Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4687 - accuracy: 0.8289
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3191 - accuracy: 0.8841
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2744 - accuracy: 0.8993
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2435 - accuracy: 0.9100
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2183 - accuracy: 0.9179
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1987 - accuracy: 0.9252
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1817 - accuracy: 0.9321
Epoch 8/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1649 - accuracy: 0.9385
Epoch 9/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1474 - accuracy: 0.9439
Epoch 10/10
313/313 [==============================] - 1s 3ms/step - loss: 0

N=16

In [7]:
import tensorflow as tf
print(tf.__version__)
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),  
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=10)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)

2.8.0
Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.5026 - accuracy: 0.8198
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3528 - accuracy: 0.8716
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3123 - accuracy: 0.8857
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2836 - accuracy: 0.8948
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2639 - accuracy: 0.9022
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2455 - accuracy: 0.9093
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2316 - accuracy: 0.9141
Epoch 8/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2182 - accuracy: 0.9189
Epoch 9/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2073 - accuracy: 0.9225
Epoch 10/10
313/313 [==============================] - 1s 3ms/step - loss: 0

N=64

In [8]:
import tensorflow as tf
print(tf.__version__)
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),  
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=10)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)

2.8.0
Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4427 - accuracy: 0.8382
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2965 - accuracy: 0.8908
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2503 - accuracy: 0.9064
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2176 - accuracy: 0.9184
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1923 - accuracy: 0.9276
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1685 - accuracy: 0.9369
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1486 - accuracy: 0.9441
Epoch 8/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1315 - accuracy: 0.9503
Epoch 9/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1146 - accuracy: 0.9553
Epoch 10/10
313/313 [==============================] - 1s 3ms/step - loss: 0


2. Remova a última convolução. Qual é o impacto?



In [9]:
import tensorflow as tf
print(tf.__version__)
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  #tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  #tf.keras.layers.MaxPooling2D(2,2),  
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=10)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)

2.8.0
Epoch 1/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3879 - accuracy: 0.8622
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2615 - accuracy: 0.9054
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2187 - accuracy: 0.9208
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1851 - accuracy: 0.9314
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1604 - accuracy: 0.9409
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1372 - accuracy: 0.9495
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1181 - accuracy: 0.9564
Epoch 8/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1004 - accuracy: 0.9638
Epoch 9/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0826 - accuracy: 0.9705
Epoch 10/10
313/313 [==============================] - 1s 2ms/step - loss: 0


3. Adicione mais uma camada convolucional. Qual é o impacto?


In [10]:
import tensorflow as tf
print(tf.__version__)
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),  
  tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),  
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=10)
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)

2.8.0
Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.6502 - accuracy: 0.7656
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4429 - accuracy: 0.8374
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3842 - accuracy: 0.8590
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3467 - accuracy: 0.8719
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3214 - accuracy: 0.8812
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3028 - accuracy: 0.8875
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2851 - accuracy: 0.8946
Epoch 8/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2713 - accuracy: 0.9004
Epoch 9/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2611 - accuracy: 0.9033
Epoch 10/10
313/313 [==============================] - 1s 3ms/step - loss: 0


4. Implemente o callback para interromper o treinamento com um determinado nível de acurácia ou perda. 

In [11]:
import tensorflow as tf
print(tf.__version__)


class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('loss')<0.4):
      print("\nAtingiu loss < 0.4, cancelando treinamento")
      self.model.stop_training = True

callbacks = myCallback()

mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images=training_images.reshape(60000, 28, 28, 1)
training_images=training_images / 255.0
test_images = test_images.reshape(10000, 28, 28, 1)
test_images=test_images/255.0
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),  
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(training_images, training_labels, epochs=10, callbacks=[callbacks])
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)

2.8.0
Epoch 1/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.5132 - accuracy: 0.8146
Epoch 2/10
1859/1875 [============================>.] - ETA: 0s - loss: 0.3542 - accuracy: 0.8713
Atingiu loss < 0.4, cancelando treinamento
313/313 [==============================] - 1s 2ms/step - loss: 0.3466 - accuracy: 0.8730
0.8730000257492065
